## Combine Two DF

### 3. Write a Pyspark program to report the first name, last name, city, and state of each person in the Person dataframe. If the address of a personId is not present in the Address dataframe, report null instead.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *  # Import the function
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.functions import col, explode, array_repeat
from pyspark.sql.window import Window
import json
spark = SparkSession.builder.getOrCreate()

In [2]:
person_Schema = StructType([
    StructField("personId", IntegerType(), True),
    StructField("lastName", StringType(), True),
    StructField("firstName", StringType(), True)
])

address_Schema = StructType([
    StructField("addressId", IntegerType(), True),
    StructField("personId", IntegerType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True)    
])

# Define data for the 'persons' table
persons_data = [
(1, 'Wang', 'Allen'),
(2, 'Alice', 'Bob')
]
# Define data for the 'addresses' table
addresses_data = [
(1, 2, 'New York City', 'New York'),
(2, 3, 'Leetcode', 'California')
]
# Create DataFrames
persons_df = spark.createDataFrame(persons_data, schema=person_Schema)  
persons_df.show()
addresses_df = spark.createDataFrame(addresses_data, schema=address_Schema)
addresses_df.show()


+--------+--------+---------+
|personId|lastName|firstName|
+--------+--------+---------+
|       1|    Wang|    Allen|
|       2|   Alice|      Bob|
+--------+--------+---------+

+---------+--------+-------------+----------+
|addressId|personId|         city|     state|
+---------+--------+-------------+----------+
|        1|       2|New York City|  New York|
|        2|       3|     Leetcode|California|
+---------+--------+-------------+----------+



In [4]:
combined_df = persons_df.join(addresses_df, persons_df.personId == addresses_df.personId, "left").select(col('firstName'), col('lastName'),col('city'),col('state'))
combined_df.show()

+---------+--------+-------------+--------+
|firstName|lastName|         city|   state|
+---------+--------+-------------+--------+
|    Allen|    Wang|         NULL|    NULL|
|      Bob|   Alice|New York City|New York|
+---------+--------+-------------+--------+



In [5]:
conditional_df = combined_df\
    .withColumn(
        'city', 
            when(col('city').isNull(), 'Unknown').otherwise(col('city'))
    ).withColumn(
        'state',
            when(col('state').isNull(), 'Unknown').otherwise(col('state'))
    )

conditional_df.show()
    

+---------+--------+-------------+--------+
|firstName|lastName|         city|   state|
+---------+--------+-------------+--------+
|    Allen|    Wang|      Unknown| Unknown|
|      Bob|   Alice|New York City|New York|
+---------+--------+-------------+--------+

